# Heat Maps Generation
This notebook allows you to generate heat maps based on population and heat data.

In [2]:
# Import necessary libraries
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx
from matplotlib.patches import Patch
import numpy as np


## Define helper functions

In [3]:
def load_data(geojson_path, pop_data_path, county_geojson_path, heat_data_path, road_data_path):
    print(f"Loading GeoJSON data from {geojson_path}")
    gdf = gpd.read_file(geojson_path)
    print(f"GeoJSON data loaded: {gdf.shape[0]} records")
    print(gdf.head())

    print(f"Loading population data from {pop_data_path}")
    pop_df = pd.read_csv(pop_data_path, dtype={'GEOID': str})
    print(f"Population data loaded: {pop_df.shape[0]} records")
    print(pop_df.head())

    heat_df = None
    if heat_data_path:
        print(f"Loading heat data from {heat_data_path}")
        heat_df = pd.read_csv(heat_data_path, dtype={'GEOID': str})
        print(f"Heat data loaded: {heat_df.shape[0]} records")
        print(heat_df.head())

    print(f"Loading county boundaries from {county_geojson_path}")
    counties_gdf = gpd.read_file(county_geojson_path)
    print(f"County boundaries loaded: {counties_gdf.shape[0]} records")
    print(counties_gdf.head())

    print(f"Loading roads data from {road_data_path}")
    roads_gdf = gpd.read_file(road_data_path)
    print(f"Roads data loaded: {roads_gdf.shape[0]} records")
    print(roads_gdf.head())

    return gdf, pop_df, heat_df, counties_gdf, roads_gdf


In [4]:
def preprocess_data(gdf, pop_df, heat_df, map_type):
    print("Ensuring the columns used for joining have the same data type and padding GEOID with leading zeros")
    gdf['GEOID'] = gdf['GEOID'].astype(str).str.zfill(11)
    pop_df['GEOID'] = pop_df['GEOID'].astype(str).str.zfill(11)
    if map_type == "heat" and heat_df is not None:
        heat_df['GEOID'] = heat_df['GEOID'].astype(str).str.zfill(11)

    print("Setting index for join")
    gdf.set_index('GEOID', inplace=True)
    pop_df.set_index('GEOID', inplace=True)
    if map_type == "heat" and heat_df is not None:
        heat_df.set_index('GEOID', inplace=True)

    print("Performing the join")
    joined_gdf = gdf.join(pop_df, how='inner')
    if map_type == "heat" and heat_df is not None:
        joined_gdf = joined_gdf.join(heat_df[['avg_90F']], how='inner')
    print(f"Joined data: {joined_gdf.shape[0]} records")
    print(joined_gdf.head())

    return joined_gdf


In [5]:
def reproject_data(joined_gdf, counties_gdf, roads_gdf):
    print("Reprojecting to Web Mercator (EPSG:3857)")
    joined_gdf = joined_gdf.to_crs(epsg=3857)
    counties_gdf = counties_gdf.to_crs(epsg=3857)
    roads_gdf = roads_gdf.to_crs(epsg=3857)

    return joined_gdf, counties_gdf, roads_gdf


In [8]:
def plot_map(county, county_gdf, counties_gdf, roads_gdf, basemap_source, label_layer, zoom, dpi, output_dir, overall_average):
    print(f"Processing county: {county}")
    county_gdf = county_gdf[county_gdf['county'] == county]
    print(f"County data: {county_gdf.shape[0]} records")

    county_gdf = county_gdf[county_gdf.is_valid & ~county_gdf.is_empty]

    if county_gdf.empty:
        print(f"No valid geometries for {county}")
        return

    latino_gdf = county_gdf[county_gdf['Neighborhood_type'] == '70+ Latino']
    white_gdf = county_gdf[county_gdf['Neighborhood_type'] == '70+ NL White']

    combined_gdf = pd.concat([latino_gdf, white_gdf])

    if combined_gdf.empty:
        print(f"No valid Latino or White 70+ neighborhoods for {county}")
        return

    county_shape = counties_gdf[counties_gdf['name'] == county]

    clipped_roads = gpd.overlay(roads_gdf, county_shape, how='intersection')

    fig, ax = plt.subplots(1, 1, figsize=(15, 15))

    county_shape.boundary.plot(ax=ax, linewidth=2.5, edgecolor='gray', zorder=3, alpha=0.45)

    below_avg = overall_average
    above_avg = overall_average

    legend_elements = []

    latino_colors = ['#fc9272', '#fb6a4a']
    white_colors = ['#6baed6', '#0570b0']

    for i, (color, label) in enumerate(zip(latino_colors, ['Below Average', 'Above Average'])):
        if label == 'Below Average':
            mask = latino_gdf['avg_90F'] < below_avg
        else:
            mask = latino_gdf['avg_90F'] > above_avg

        if not latino_gdf[mask].empty:
            latino_gdf[mask].plot(ax=ax, color=color, edgecolor='none', linewidth=0.5, alpha=0.6)
            legend_elements.append(Patch(facecolor=color, edgecolor='none', label=f'{label} (Latino) ({round(latino_gdf[mask]["avg_90F"].min())}-{round(latino_gdf[mask]["avg_90F"].max())} days)', alpha=0.6))

    for i, (color, label) in enumerate(zip(white_colors, ['Below Average', 'Above Average'])):
        if label == 'Below Average':
            mask = white_gdf['avg_90F'] < below_avg
        else:
            mask = white_gdf['avg_90F'] > above_avg

        if not white_gdf[mask].empty:
            white_gdf[mask].plot(ax=ax, color=color, edgecolor='none', linewidth=0.5, alpha=0.6)
            legend_elements.append(Patch(facecolor=color, edgecolor='none', label=f"{label} (White) ({round(white_gdf[mask]['avg_90F'].min())}-{round(white_gdf[mask]['avg_90F'].max())} days)", alpha=0.6))

    if pd.notna(overall_average):
        legend_elements.append(Patch(facecolor='none', edgecolor='none', label=f'Overall Average: {round(overall_average)} days'))

    no_data_gdf = combined_gdf[combined_gdf['avg_90F'].isna()]
    print(f"No data records: {no_data_gdf.shape[0]}")
    no_data_gdf.plot(ax=ax, color='#d9d9d9', edgecolor='lightgray', linewidth=0.5, alpha=0.6)

    if legend_elements:
        legend_title = 'Average Number of Days with 90°F+'
        legend = ax.legend(handles=legend_elements, loc='upper right', title=legend_title)
        legend.set_zorder(10)

    ctx.add_basemap(ax, source=basemap_source, zoom=zoom)
    ctx.add_basemap(ax, source=label_layer, zoom=zoom)

    ax.set_aspect('equal')
    ax.set_axis_off()

    output_subdir = os.path.join(output_dir, 'combined_maps')
    os.makedirs(output_subdir, exist_ok=True)

    output_path = os.path.join(output_subdir, f'{county}_heat_map_combined.png')
    print(f"Saving map to {output_path}")

    plt.savefig(output_path, bbox_inches='tight', pad_inches=0.1, dpi=dpi)
    plt.close()

    print(f"Map for {county} saved to {output_path}")


## Set up paths and directories

In [9]:
geojson_path = 'inputs/geojson/ca_census_tracts.geojson'
ca_counties_path = 'inputs/geojson/ca_counties_simplified.geojson'
pop_data_path = 'inputs/tract_level_data.csv'
heat_data_path = 'inputs/heat_data.csv'
output_dir = 'output/test_heat_maps'
roads_path = 'inputs/geojson/ca_primary_secondary_roads.geojson'
os.makedirs(output_dir, exist_ok=True)


## Specify the county to generate the map for

In [10]:
specific_county = 'Alameda'  # Change this to the desired county name

## Load and preprocess data

In [11]:
gdf, pop_df, heat_df, counties_gdf, roads_gdf = load_data(geojson_path, pop_data_path, ca_counties_path, heat_data_path, roads_path)
joined_gdf = preprocess_data(gdf, pop_df, heat_df, map_type="heat")
joined_gdf, counties_gdf, roads_gdf = reproject_data(joined_gdf, counties_gdf, roads_gdf)
overall_average = joined_gdf['avg_90F'].mean()
print(f"Overall average: {overall_average}")


Loading GeoJSON data from inputs/geojson/ca_census_tracts.geojson
GeoJSON data loaded: 9104 records
  STATEFP COUNTYFP TRACTCE               GEOIDFQ        GEOID   NAME  \
0      06      077  005127  1400000US06077005127  06077005127  51.27   
1      06      077  003406  1400000US06077003406  06077003406  34.06   
2      06      077  004402  1400000US06077004402  06077004402  44.02   
3      06      077  001700  1400000US06077001700  06077001700     17   
4      06      077  000401  1400000US06077000401  06077000401   4.01   

             NAMELSAD STUSPS          NAMELSADCO  STATE_NAME LSAD    ALAND  \
0  Census Tract 51.27     CA  San Joaquin County  California   CT  1960015   
1  Census Tract 34.06     CA  San Joaquin County  California   CT   839414   
2  Census Tract 44.02     CA  San Joaquin County  California   CT  4346359   
3     Census Tract 17     CA  San Joaquin County  California   CT  1685934   
4   Census Tract 4.01     CA  San Joaquin County  California   CT  1045658   

## Generate Combined Heat Map

In [13]:
plot_map(specific_county, joined_gdf, counties_gdf, roads_gdf, ctx.providers.CartoDB.Positron, ctx.providers.CartoDB.PositronOnlyLabels, zoom=10, dpi=300, output_dir=output_dir, overall_average=overall_average)
expected_output_file = os.path.join(output_dir, 'combined_maps_nb', f'{specific_county}_heat_map_combined.png')
print(f"Expected output file: {expected_output_file}")
print(f"Contents of output directory: {os.listdir(os.path.join(output_dir, 'combined_maps'))}")
assert os.path.exists(expected_output_file), "Combined heat map was not generated successfully"

Processing county: Alameda
County data: 378 records
No data records: 0


C:\Users\hikou\AppData\Local\Temp\ipykernel_28728\3693490717.py:62: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  no_data_gdf.plot(ax=ax, color='#d9d9d9', edgecolor='lightgray', linewidth=0.5, alpha=0.6)


Saving map to output/test_heat_maps\combined_maps\Alameda_heat_map_combined.png
Map for Alameda saved to output/test_heat_maps\combined_maps\Alameda_heat_map_combined.png
Expected output file: output/test_heat_maps\combined_maps_nb\Alameda_heat_map_combined.png
Contents of output directory: ['Alameda_heat_map_combined.png', 'Contra Costa_heat_map_combined.png', 'Fresno_heat_map_combined.png', 'Imperial_heat_map_combined.png', 'Kern_heat_map_combined.png', 'Kings_heat_map_combined.png', 'Los Angeles_heat_map_combined.png', 'Madera_heat_map_combined.png', 'Merced_heat_map_combined.png', 'Monterey_heat_map_combined.png', 'Orange_heat_map_combined.png', 'Riverside_heat_map_combined.png', 'San Bernardino_heat_map_combined.png', 'San Diego_heat_map_combined.png', 'San Joaquin_heat_map_combined.png', 'San Mateo_heat_map_combined.png', 'Santa Barbara_heat_map_combined.png', 'Santa Clara_heat_map_combined.png', 'Santa Cruz_heat_map_combined.png', 'Stanislaus_heat_map_combined.png', 'Tulare_heat

AssertionError: Combined heat map was not generated successfully